In [2]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

random_seed=42

In [3]:
data = pd.read_csv('dataSUPER (1).csv',
                   low_memory=False,
                   lineterminator='\r').drop(['id', 'Apartment type', 'Region'], axis=1)

In [4]:
data['Metro station'] = data['Metro station'][:-1].apply(lambda x: x[1:] if x[0] == ' ' else x)
data = data[(data['Area'] < 500)]
data = data[(data['Kitchen area'] < 50)]
data = data[(data['Minutes to metro'] < 40)]
data = data[(data['Floor'] < 50)]
data = data[(data['Number of floors'] < 60)]
data = data[(data['Living area'] < 230)]


In [5]:
cat_features = []

for col in data.columns:
    if data[col].dtype == 'object':
        cat_features.append(col)
        data[col] = data[col].fillna('-1')
        
        
', '.join(cat_features)

X, y = data.drop('Price', axis=1), data['Price']
y.min(), y.max()

(1150000.0, 1003158200.0)

In [6]:
X = X[(y>np.percentile(y, 5)) & (y<np.percentile(y, 95))]
y = y[(y>np.percentile(y, 5)) & (y<np.percentile(y, 95))]
y.min(), y.max()

(4850000.0, 211800000.0)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=random_seed)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5, shuffle=True, random_state=random_seed)

## Train

In [8]:
model = CatBoostRegressor(n_estimators=4000, loss_function='MAE')
model.fit(X_train, y_train, eval_set=(X_valid, y_valid),cat_features=cat_features, verbose=0, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [9]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,Area,39.701312
1,Metro station,19.598556
2,Renovation,8.342790
3,Living area,8.014690
4,Number of floors,7.595888
5,Kitchen area,5.985968
6,Minutes to metro,3.778725
7,Floor,3.562814
8,Number of rooms,3.419257
